In [1]:
import sqlalchemy as sq
import geopandas as gpd  # type: ignore
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os, sys, calendar
from aggregateErgot import calcUIDs

sys.path.append("../")
from Shared.DataService import DataService
from Shared.GenericQueryBuilder import GenericQueryBuilder

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_1120077/326827117.py:2: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (callin

In [2]:
load_dotenv()
PG_DB = os.getenv("POSTGRES_DB")
PG_ADDR = os.getenv("POSTGRES_ADDR")
PG_PORT = os.getenv("POSTGRES_PORT")
PG_USER = os.getenv("POSTGRES_USER")
PG_PW = os.getenv("POSTGRES_PW")

if (
    PG_DB is None
    or PG_ADDR is None
    or PG_PORT is None
    or PG_USER is None
    or PG_PW is None
):
    raise ValueError("Environment variables not set")

In [3]:
def pullIndividualErgotSampleData(conn: sq.engine.Connection) -> pd.DataFrame:
    # pulling weather station data from the database
    weatherDataQuery = sq.text(
        """
        SELECT * FROM public.ergot_sample
        """
    )

    return pd.read_sql(weatherDataQuery, conn)

In [4]:
def pullAggErgotData(conn: sq.engine.Connection) -> pd.DataFrame:
    # pulling weather station data from the database
    weatherDataQuery = sq.text(
        """
        SELECT * FROM public.agg_ergot_sample_v2
        """
    )

    return pd.read_sql(weatherDataQuery, conn)

In [5]:
def createErgotFeatEngTableV1(db, tablename: str):
    query = sq.text(
        f"""
        CREATE TABLE {tablename} (
            year                        INT,
            province                    VARCHAR(2),
            crop_district               INT,
            incidence                   BOOL,
            severity                    FLOAT,
            district                    INT,
            downgrade                   BOOL,
            severity_bin_quan           INT,
            severity_bin_arb            INT,

            CONSTRAINT PK_{tablename} PRIMARY KEY(year, district)
        );
        COMMIT;
        """
    )

    db.execute(query)

In [6]:
def calculateDowngradeColumn(df: pd.DataFrame) -> pd.DataFrame:
    DOWNGRADE_THRESHOLD = 0.04
    df["downgrade"] = False
    df.loc[df["severity"] >= DOWNGRADE_THRESHOLD, "downgrade"] = True
    return df

In [7]:
def calculateSeverityBinQuan(df: pd.DataFrame) -> pd.DataFrame:
    # quantiles only on severities > 0
    df["severity_bin_quan"] = 0
    df.loc[df["severity"] > 0, "severity_bin_quan"] = pd.qcut(
        df.loc[df["severity"] > 0]["severity"], 4, labels=False
    )
    return df

In [8]:
def calculateSeverityBinArbitrary(df: pd.DataFrame) -> pd.DataFrame:
    df["severity_bin_arb"] = 0
    df.loc[df["severity"] >= 0.02, "severity_bin_arb"] = 1
    df.loc[df["severity"] >= 0.04, "severity_bin_arb"] = 2
    df.loc[df["severity"] >= 0.08, "severity_bin_arb"] = 3
    return df

In [9]:
def main():
    TABLENAME = "ergot_sample_feat_eng"

    db = DataService(PG_DB, PG_ADDR, int(PG_PORT), PG_USER, PG_PW)
    conn = db.connect()

    ergotDf = pullIndividualErgotSampleData(conn)
    ergotDf = calcUIDs(ergotDf)
    ergotDf = calculateDowngradeColumn(ergotDf)
    ergotDf = calculateSeverityBinQuan(ergotDf)
    ergotDf = calculateSeverityBinArbitrary(ergotDf)

    try:
        queryBuilder = GenericQueryBuilder()
        request = sq.text(queryBuilder.tableExistsReq(TABLENAME))
        tableExists = queryBuilder.readTableExists(db.execute(request))

        if not tableExists:
            createErgotFeatEngTableV1(db)

        ergotDf.to_sql(
            TABLENAME, conn, schema="public", if_exists="append", index=False
        )
    except Exception as e:
        print("An error occurred while writing to the database {}".format(e))
        raise e

    db.cleanup()

In [10]:
if __name__ == "__main__":
    main()